In [ ]:
import cv2
from cv2 import imread
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
from cv2 import rectangle

import numpy as np
import pickle

from PIL import Image, ImageDraw
from IPython import display

In [ ]:
face_cascade = CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
eye_cascade = CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [ ]:
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
image_dir = os.path.join(BASE_DIR, "images")

y_labels = []
x_train = []

current_id = 0
labels_id = {}
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith('jpg'):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path))
            pil_image = Image.open(path).convert("L")

            size = (550, 550)
            pil_image = pil_image.resize(size, Image.ANTIALIAS)

            image_array = np.array(pil_image, "uint8")
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            if not label in labels_id:
                labels_id[label] = current_id
                current_id += 1

            id_ = labels_id[label]
            for (x, y, w, h) in faces:
                roi = image_array[y:y+h,  x:x+w] # face from frame in gray
                x_train.append(roi)
                y_labels.append(id_)

In [ ]:
recognizer.train(x_train, np.array(y_labels))

In [ ]:
# recognizer.save("trainer.yml")
recognizer.read("trainer.yml")

In [ ]:
# with open('labels.pickle', 'wb') as file:
#     pickle.dump(labels_id, file)

with open('labels.pickle', 'rb') as file:
    og_labels = pickle.load(file)
    labels = {v:k for k,v in og_labels.items()}

In [ ]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h,  x:x+w] # face from frame in gray
        roi_color = frame[y:y+h,  x:x+w] # face from frame in color

        id_, conf = recognizer.predict(roi_gray)

        if conf >= 45:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

        # saving face (just in some case)
        img_item = 'my_face.png'
        cv2.imwrite(img_item, roi_color)

        color = (255, 0, 0) #BGR
        stroke = 5
        width_frame = x + w
        height_frame = y + h
        cv2.rectangle(frame, (x, y), (width_frame, height_frame), color, stroke)

    eyes = eye_cascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in eyes:
        roi_gray = gray[y:y+h,  x:x+w]
        color = (255, 0, 0) #BGR
        stroke = 5
        width_frame = x + w
        height_frame = y + h
        cv2.rectangle(frame, (x, y), (width_frame, height_frame), color, stroke)


    # print(roi_color.shape)
    # h, w, _ = frame.shape
    # img = cv2.cvtColor(frame, cv2.COLOR_BGR2BGR555)
    # img  = cv2.resize(img, (640, 480))
    # img = np.transpose(img, [2, 0, 1])
    # img = np.expand_dims(img, axis=0)
    # img = img.astype(np.float32)
    # img *= (255.0/img.max())

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
